In [2]:
import pandas as pd

In [3]:
calendar=pd.read_csv(r"C:\Users\priya\SparePartsInventory\data\Calendar_new.csv")
sales=pd.read_csv(r"C:\Users\priya\SparePartsInventory\notebooks\Sampled_sales.csv")
sell_prices=pd.read_csv(r"C:\Users\priya\SparePartsInventory\data\sell_prices.csv")


In [4]:
# Drop unnamed columns (usually index columns auto-saved)
sales = sales.loc[:, ~sales.columns.str.contains('^Unnamed')]
calendar = calendar.loc[:, ~calendar.columns.str.contains('^Unnamed')]
sell_prices=sell_prices.loc[:, ~sell_prices.columns.str.contains('^Unnamed')]

# Also drop any column literally named 'index'
sales= sales.drop(columns=['index'], errors='ignore')
calendar = calendar.drop(columns=['index'], errors='ignore')
sell_prices = sell_prices.drop(columns=['index'], errors='ignore')
# Optional: Reset index if previously saved with index
sales.reset_index(drop=True, inplace=True)
calendar.reset_index(drop=True, inplace=True)
sell_prices.reset_index(drop=True, inplace=True)

In [5]:
merged_df = pd.merge(sales, calendar, how='left', on='d')
# Check if the first column is just an index
if merged_df.columns[0] == merged_df.index.name or merged_df.columns[0].startswith('Unnamed') or merged_df.columns[0] == '0':
    merged_df = merged_df.iloc[:, 1:]  # Drop first column

# Reset index just to be safe
merged_df.reset_index(drop=True, inplace=True)
merged_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,units_sold,date,wm_yr_wk,...,snap_flag,week_of_month,season,day_of_year,is_payday,is_working_Day,event_in_3days,event_in_7days,is_month_start,is_month_end
0,FOODS_3_180_CA_1_validation,FOODS_3_180,FOODS_3,FOODS,CA_1,CA,d_1,0,29-01-2011,11101,...,0,5,Winter,29,0,0,0,0,0,0
1,HOUSEHOLD_2_383_CA_3_validation,HOUSEHOLD_2_383,HOUSEHOLD_2,HOUSEHOLD,CA_3,CA,d_1,2,29-01-2011,11101,...,0,5,Winter,29,0,0,0,0,0,0
2,FOODS_3_409_CA_3_validation,FOODS_3_409,FOODS_3,FOODS,CA_3,CA,d_1,0,29-01-2011,11101,...,0,5,Winter,29,0,0,0,0,0,0
3,FOODS_1_097_CA_2_validation,FOODS_1_097,FOODS_1,FOODS,CA_2,CA,d_1,0,29-01-2011,11101,...,0,5,Winter,29,0,0,0,0,0,0
4,HOBBIES_1_272_TX_2_validation,HOBBIES_1_272,HOBBIES_1,HOBBIES,TX_2,TX,d_1,0,29-01-2011,11101,...,0,5,Winter,29,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583460,HOUSEHOLD_1_505_TX_2_validation,HOUSEHOLD_1_505,HOUSEHOLD_1,HOUSEHOLD,TX_2,TX,d_999,6,23-10-2013,11339,...,0,4,Fall,296,0,1,0,0,0,0
583461,HOBBIES_1_120_WI_3_validation,HOBBIES_1_120,HOBBIES_1,HOBBIES,WI_3,WI,d_999,1,23-10-2013,11339,...,0,4,Fall,296,0,1,0,0,0,0
583462,FOODS_3_075_TX_1_validation,FOODS_3_075,FOODS_3,FOODS,TX_1,TX,d_999,0,23-10-2013,11339,...,0,4,Fall,296,0,1,0,0,0,0
583463,HOUSEHOLD_2_057_TX_2_validation,HOUSEHOLD_2_057,HOUSEHOLD_2,HOUSEHOLD,TX_2,TX,d_999,0,23-10-2013,11339,...,0,4,Fall,296,0,1,0,0,0,0


In [6]:
df_final = pd.merge(
    merged_df,
    sell_prices,
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='inner'
)

In [7]:
df_final.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'units_sold', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'is_event', 'is_weekend', 'snap_flag',
       'week_of_month', 'season', 'day_of_year', 'is_payday', 'is_working_Day',
       'event_in_3days', 'event_in_7days', 'is_month_start', 'is_month_end',
       'sell_price'],
      dtype='object')

In [8]:
df_final['revenue'] = df_final['units_sold'] * df_final['sell_price']


In [9]:
# Rename columns for Spare Parts Inventory Simulation
df_final.rename(columns={
    'item_id': 'part_id',
    'store_id': 'location_id',
    'sell_price': 'unit_cost',
    'units_sold': 'daily_demand_units',
    'dept_id': 'part_type',
    'cat_id': 'category',
    'state_id': 'region',
    'wm_yr_wk': 'year_week',
    'weekday': 'day_name',
    'wday': 'day_num'
}, inplace=True)


In [10]:
for col in df_final:
    print(df_final[col].value_counts())

id
HOUSEHOLD_2_284_CA_3_validation    1913
FOODS_2_207_WI_3_validation        1913
HOBBIES_1_364_CA_3_validation      1913
FOODS_2_181_TX_3_validation        1913
FOODS_3_331_TX_1_validation        1913
                                   ... 
FOODS_2_223_CA_2_validation         338
FOODS_2_346_CA_2_validation         338
FOODS_2_096_CA_2_validation         275
HOUSEHOLD_1_441_CA_1_validation     240
FOODS_3_192_WI_2_validation         191
Name: count, Length: 305, dtype: int64
part_id
HOBBIES_1_100      3826
FOODS_3_309        3784
FOODS_2_303        3770
HOBBIES_2_100      3763
FOODS_2_184        3602
                   ... 
FOODS_2_223         338
FOODS_2_346         338
FOODS_2_096         275
HOUSEHOLD_1_441     240
FOODS_3_192         191
Name: count, Length: 290, dtype: int64
part_type
FOODS_3        124428
FOODS_2         68290
HOBBIES_1       63156
HOUSEHOLD_2     61154
HOUSEHOLD_1     56729
HOBBIES_2       41415
FOODS_1         26571
Name: count, dtype: int64
category
FOODS   

In [11]:
category_map = {
    "FOODS": "Fast_Moving",
    "HOUSEHOLD": "Regular",
    "HOBBIES": "Optional"
}
df_final["part_class"] = df_final["category"].map(category_map)


In [12]:
df_final.isnull().sum().sort_values(ascending=False)

event_type_2          440846
event_name_2          440846
event_type_1          406380
event_name_1          406380
category                   0
part_type                  0
part_id                    0
id                         0
location_id                0
region                     0
d                          0
daily_demand_units         0
day_num                    0
day_name                   0
year_week                  0
date                       0
month                      0
year                       0
snap_CA                    0
snap_TX                    0
snap_WI                    0
is_event                   0
is_weekend                 0
snap_flag                  0
week_of_month              0
season                     0
day_of_year                0
is_payday                  0
is_working_Day             0
event_in_3days             0
event_in_7days             0
is_month_start             0
is_month_end               0
unit_cost                  0
revenue       

In [13]:
df_final.drop(["event_name_2", "event_type_2"], axis=1, inplace=True)


In [14]:
df_final["event_name_1"] = df_final["event_name_1"].fillna("No Event")
df_final["event_type_1"] = df_final["event_type_1"].fillna("No Event")

In [15]:
df_final.isnull().sum().sort_values(ascending=False)

id                    0
part_id               0
part_type             0
category              0
location_id           0
region                0
d                     0
daily_demand_units    0
date                  0
year_week             0
day_name              0
day_num               0
month                 0
year                  0
event_name_1          0
event_type_1          0
snap_CA               0
snap_TX               0
snap_WI               0
is_event              0
is_weekend            0
snap_flag             0
week_of_month         0
season                0
day_of_year           0
is_payday             0
is_working_Day        0
event_in_3days        0
event_in_7days        0
is_month_start        0
is_month_end          0
unit_cost             0
revenue               0
part_class            0
dtype: int64

In [23]:
pd.set_option('display.max_rows', None)

print(df_final['d'].value_counts().sort_index())

d
d_1       100
d_10      109
d_100     139
d_1000    243
d_1001    243
d_1002    243
d_1003    243
d_1004    243
d_1005    243
d_1006    243
d_1007    243
d_1008    243
d_1009    243
d_101     139
d_1010    243
d_1011    243
d_1012    243
d_1013    243
d_1014    243
d_1015    243
d_1016    243
d_1017    243
d_1018    243
d_1019    243
d_102     139
d_1020    243
d_1021    243
d_1022    243
d_1023    243
d_1024    243
d_1025    243
d_1026    243
d_1027    243
d_1028    243
d_1029    243
d_103     139
d_1030    243
d_1031    243
d_1032    243
d_1033    243
d_1034    243
d_1035    243
d_1036    243
d_1037    243
d_1038    243
d_1039    243
d_104     139
d_1040    243
d_1041    243
d_1042    243
d_1043    243
d_1044    245
d_1045    245
d_1046    245
d_1047    245
d_1048    245
d_1049    245
d_105     139
d_1050    245
d_1051    247
d_1052    247
d_1053    247
d_1054    247
d_1055    247
d_1056    247
d_1057    247
d_1058    248
d_1059    248
d_106     139
d_1060    248
d_1061    248
d_10

In [17]:
df_final.to_csv("SparePartsInventory.csv", index=False)
